In [ ]:
import os
import pandas as pd
from SpatioTemporal_calculation import compute_spatiotemporal_variables
from gait_events import gait_events_HC_JA
#from gait_events import recortar_eventos
import importlib
import gait_events
importlib.reload(gait_events)
sampling_rate = 200 # despues del downsampling 
#import importlib
#import gait_events
#importlib.reload(gait_events)

In [ ]:
# Función para cargar todos los archivos de un paciente (18 tomas)
def load_patient_data(patient_folder):
    df_list = []
    file_list = []
    
    for d in ['D01', 'D02']:
        for b in ['B01', 'B02', 'B03']:
            for t in ['T01', 'T02', 'T03']:
                file_name = f"{patient_folder}/S002_G01_{d}_{b}_{t}.csv"
                if os.path.exists(file_name):
                    try:
                        df = pd.read_csv(file_name)
                        if df.empty:
                            print(f"Archivo vacío: {file_name}")
                            continue
                        df_list.append(df)
                        file_list.append(file_name)
                    except Exception as e:
                        print(f"Error al leer {file_name}: {e}")
                else:
                    print(f"Archivo no encontrado: {file_name}")
    return df_list, file_list

# Define la carpeta del paciente 
patient_folder = "./young adults (19–35 years old)/S002"  

# Cargar todos los DataFrames y nombres de archivo
df_list, file_list = load_patient_data(patient_folder)

# Lista para almacenar los resultados de cada toma
results = []

for i, df in enumerate(df_list):
    trial_name = file_list[i]
    print(f"\nProcesando la toma: {trial_name}")
    
    # Paso 1: Detección de eventos de marcha
    try:
        print("  Ejecutando la detección de gait events...")
        heel_strike_R, heel_strike_L, toe_off_R, toe_off_L = gait_events_HC_JA(df)
        print(f"  Detección de gait events exitosa para {trial_name}:")
        #print(f"    Heel Strikes Right: {len(heel_strike_R)}")
        #print(f"    Heel Strikes Left: {len(heel_strike_L)}")
        #print(f"    Toe Offs Right: {len(toe_off_R)}")
        #print(f"    Toe Offs Left: {len(toe_off_L)}")
    except Exception as e:
        print(f"  Error en la detección de gait events para {trial_name}: {e}")
        continue  # Salta esta toma si falla la detección de eventos

    # Verifica si se detectaron suficientes eventos
    if not heel_strike_R or not toe_off_R:
        print(f"  Se omite la toma {trial_name} por eventos insuficientes.")
        continue

    # Paso 2: Cálculo de variables espaciotemporales
    try:
        print("  Ejecutando el cálculo de variables espaciotemporales...")
        spatiotemporal_df, mean_df, std_df = compute_spatiotemporal_variables(df, heel_strike_R, heel_strike_L, toe_off_R, toe_off_L, sampling_rate)
        if mean_df.empty:
            print(f"  Se omite la toma {trial_name} por resultados espaciotemporales vacíos.")
            continue
        print(f"  Cálculo de variables espaciotemporales completado para {trial_name}.")
    except Exception as e:
        print(f"  Error en el cálculo espaciotemporal para {trial_name}: {e}")
        continue

    # Agrega una columna para identificar la toma
    mean_df['Trial'] = os.path.basename(trial_name)
    results.append(mean_df)

# Combina los resultados de todas las tomas en un solo DataFrame
if not results:
    print("No se generaron resultados para ninguna toma.")
else:
    final_df = pd.concat(results, ignore_index=True)
    print("\nVista previa del DataFrame final:")
    print(final_df.head())




Procesando la toma: ./young adults (19–35 years old)/S002/S002_G01_D01_B01_T01.csv
  Ejecutando la detección de gait events...
  Detección de gait events exitosa para ./young adults (19–35 years old)/S002/S002_G01_D01_B01_T01.csv:
    Heel Strikes Right: 201
    Heel Strikes Left: 200
    Toe Offs Right: 200
    Toe Offs Left: 199
  Ejecutando el cálculo de variables espaciotemporales...
DEBUG: left_stance_time shape: (199,)
DEBUG: left_stride_time shape: (199,)
DEBUG: right_stance_time shape: (199,)
DEBUG: right_stride_time shape: (199,)
DEBUG: left_swing_time shape: (199,)
DEBUG: right_swing_time shape: (199,)
DEBUG: double_support_time shape: (199,)
DEBUG: single_support_time shape: (199,)
DEBUG: pct_double shape: (199,)
DEBUG: pct_single shape: (199,)
DEBUG: left_stride_length shape: (199,)
DEBUG: right_stride_length shape: (199,)
DEBUG: left_step_width shape: (199,)
DEBUG: left_step_length shape: (199,)
DEBUG: right_step_width shape: (199,)
DEBUG: right_step_length shape: (199,)


In [ ]:
spatio_df